# Dependencies 

In [1]:
# Dependencies: 
import os
import pandas as pd
import numpy as np 
import pickle
import time
import pymongo
from bs4 import BeautifulSoup
from splinter import Browser 
import requests
from pprint import pprint

# Portland MLS 
    -Navigate to homepage
    -Collect the number of pages and number of listing per page
    -Scrape the href to all the listings shown

In [ ]:
# Initialize Splinter
executable_path = {'executable_path': '/Users/daniellove/Downloads/chromedriver'}
browser = Browser('chrome', **executable_path, headless=True)

# Directing splinter to webpage
portlandMLS_url = 'https://www.portlandmlsdirect.com/cgi-bin/real?pge=newsearch&state=na&widget=true&sortby=price&area=Portland&price_lo=0&price_hi=100000000&tot_bed_lo=0&tot_bath_lo=0&htype=ALL'
browser.visit(portlandMLS_url)

In [3]:
# Create variables
tracker = 0
link_list = []

# Create BeautifulSoup object
soupy = BeautifulSoup(browser.html, 'html.parser')

# Identify the number of pages of houses
number_of_pages = soupy.find(id='ia_btn_text').text
page_count = int(number_of_pages[5:])
print(page_count)

# Look going through all the pages of listings
for p in range(1, page_count):
    tracker += 1 
    
    # Identify all items in the gridview class
    viewgrid_all = soupy.find_all('div', class_='viewgrid')
    last_item = viewgrid_all[-1].get('id')
    viewgrid_count = int(last_item[8:]) + 1
    
    # Loop going through each home listing on a page
    for g in range(1, viewgrid_count):
        
        # Create BeautifulSoup object
        soupy = BeautifulSoup(browser.html, 'html.parser')
        
        # Click on each grid item
        elems = soupy.find('div', {'id':(f'viewgrid{g}')}).find('a')
        start_html = "https://www.portlandmlsdirect.com/"  
        link = start_html + elems['href']
        link_list.append(link)
    
    # Go to next page
    browser.find_by_id("ia_btn_next").click()
    print(f'Completion Percentage: {round((tracker/page_count)*100, 1)}%')
    
print(len(link_list))
print(link_list)

52
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
1836
['https://www.portlandmlsdirect.com//property_listing/20692749/mlsname/RMLS/ltype/res/3157-NE-MARINE-DR-Portland-OR-97035', 'https://www.portlandmlsdirect.com//property_listing/20219978/mlsname/RMLS/ltype/res/17452-NE-GLISAN-ST--7-Portland-OR-97230', 'https://www.portlandmlsdirect.com//property_listing/19649552/mlsname/RMLS/ltype/res/23524-NW-St-Helens-RD-S19-Portland-OR-97231', 'https://www.portlandmlsdirect.com//property_listing/19015047/mlsname/RMLS/ltype/res/9034-SE-78TH-PL-Portland-OR-97206', 'https://www.portlandmlsdirect.com//property_listing/20517159/mlsname/RMLS/ltype/res/16000-SE-POWELL-BLVD-75-Portland-OR-97236', 'https://www.portlandmlsdirect.com//property_listing/20517298/mlsname/RMLS/ltype/res/12846-SE-RAMONA-ST-6-Portland-OR-97236', 'https://www.portlandmlsdirect.com//property_listing/20245155/mlsname/RMLS/ltype/res/7720-

In [4]:
# Check all links are unique
link_check = pd.DataFrame(link_list)
len(link_check[0].unique())
len(link_check[0])

1836

# Scrape Housing Data for each listing
    -address
    -price
    -bedrooms
    -bathrooms
    -square footage
    -year built
    -lot size
    -neighborhood
    -schools (elementary, middle and high)
    -county
    -city
    -zipcode

In [6]:
# Define variables
list_home_dict = []
loading_error_links = []
error_count = 0
bath = "BATH"
acres = "ACRES"
neighbor = "Neighborhood"

# Loop to iterate through all listing links
for links in link_list: 
    try:
        # Navigate to webpages for each home
        browser.visit(links)
        soup = BeautifulSoup(browser.html, 'html.parser')

        # To categorize homes with 0 bedrooms
        if bath in soup.find_all('div', class_="lineitem")[0].text:
            # Scraping primary data 
            address = soup.find('div', id="ia_address").text.replace("\n","").replace("\t  ", "")
            price = int(soup.find('div', id="ia_price").text.strip().replace('$', '').replace(',',''))
            bedrooms = 0 
            bathrooms = int(soup.find_all('div', class_="lineitem")[0].text[0])
            square_feet = int(soup.find_all('div', class_="lineitem")[1].text.replace(
                                                            'SQFT', '').replace(',', ''))
        
        else:
            # Scraping primary data
            address = soup.find('div', id="ia_address").text.replace("\n","").replace("\t  ", "")
            price = int(soup.find('div', id="ia_price").text.strip().replace('$', '').replace(',',''))
            bedrooms = int(soup.find_all('div', class_="lineitem")[0].text[0])
            bathrooms = float(soup.find_all('div', class_="lineitem")[1].text[0])
            square_feet = int(soup.find_all('div', class_="lineitem")[2].text.replace(
                                                            'SQFT', '').replace(',', ''))
        
        home_type = soup.find_all('div', id="PropDetailItem")[5].text[12:].replace('\n', '')
        built = int(soup.find_all('div', id="PropDetailItem")[3].text[12:])
        
        # Deals with pages that don't mention lot size
        if acres in soup.find_all('div', class_="lineitem")[4].text:
            lot_size = float(soup.find_all('div', class_="lineitem")[4].text.replace('ACRES', ''))

        else:
            lot_size = np.NaN 

        # Deals with pages that don't mention neighborhood
        if neighbor == soup.find_all('div', id='areaitemTitle')[3].text:
            neighborhood = soup.find_all('div', id='areaitemValue')[3].text

        else:
            neighborhood = "unknown"
            
        county = soup.find_all('div', id='areaitemValue')[0].text
        city = soup.find_all('div', id='areaitemValue')[1].text
        zipcode = soup.find_all('div', id='areaitemValue')[2].text

        # Schools data
        HS = soup.find_all('div', id="PropDetailTitle")[0].find_all_next('div')[10].text
        MS = soup.find_all('div', id="PropDetailTitle")[0].find_all_next('div')[6].text
        ES = soup.find_all('div', id="PropDetailTitle")[0].find_all_next('div')[2].text

        # Create dictionary to hold data collected
        home_dict = {
                'address':address,
                'price':price,
                'home_type':home_type,
                'bedrooms':bedrooms,
                'bathrooms':bathrooms,
                'square_feet':square_feet,
                'built':built,
                'lot_size':lot_size,
                'neighborhood':neighborhood,
                'county':county,
                'city':city,
                'zipcode':zipcode,
                'high_school':HS,
                'middle_school':MS,
                'elementary_school':ES
            }
        print(home_dict)

        # Append dictionaries to a list
        list_home_dict.append(home_dict)
    
    # Handles errors and collect links which data was able to be pulled
    except Exception as e:
        print('-------------')
        loading_error_links.append(links)
        error_count += 1
        print(error_count, e)
        
        pass

browser.quit()

{'address': '3157 NE MARINE DR, Portland OR 97035', 'price': 65000, 'home_type': 'Floating Home - 1 Story', 'bedrooms': 1, 'bathrooms': 1.0, 'square_feet': 800, 'built': 1964, 'lot_size': nan, 'neighborhood': 'unknown', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97035', 'high_school': 'Current Price:', 'middle_school': 'Jefferson', 'elementary_school': 'Faubion'}
{'address': '17452 NE GLISAN ST #7, Portland OR 97230', 'price': 72000, 'home_type': 'Manufactured - Double Wide Manufact', 'bedrooms': 2, 'bathrooms': 2.0, 'square_feet': 1152, 'built': 1988, 'lot_size': nan, 'neighborhood': 'unknown', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97230', 'high_school': 'Reynolds', 'middle_school': 'Reynolds', 'elementary_school': 'Hartley'}
-------------
1 list index out of range
{'address': '9034 SE 78TH PL, Portland OR 97206', 'price': 79950, 'home_type': 'Manufactured - Double Wide Manufact', 'bedrooms': 3, 'bathrooms': 2.0, 'square_feet': 1344, 'built': 1997, 'lot_si

{'address': '1134 SW JEFFERSON ST 604, Portland OR 97201', 'price': 185000, 'home_type': 'Condo - Studio', 'bedrooms': 1, 'bathrooms': 1.0, 'square_feet': 481, 'built': 2000, 'lot_size': nan, 'neighborhood': 'CORNERSTONE', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97201', 'high_school': 'Lincoln', 'middle_school': 'West Sylvan', 'elementary_school': 'Chapman'}
{'address': '7522 SW BARNES RD F, Portland OR 97225', 'price': 185000, 'home_type': 'Condo - Tri Level', 'bedrooms': 1, 'bathrooms': 1.0, 'square_feet': 624, 'built': 1977, 'lot_size': nan, 'neighborhood': 'unknown', 'county': 'Washington', 'city': 'Portland', 'zipcode': '97225', 'high_school': 'Beaverton', 'middle_school': 'Cedar Park', 'elementary_school': 'W Tualatin View'}
{'address': '173 NE Bridgeton RD 18, Portland OR 97211', 'price': 185000, 'home_type': 'Floating Home - Bungalow', 'bedrooms': 2, 'bathrooms': 2.0, 'square_feet': 1300, 'built': 1979, 'lot_size': nan, 'neighborhood': 'BRIDGETON', 'county': 'Mul

{'address': '606 NW NAITO PKWY A21, Portland OR 97209', 'price': 209800, 'home_type': 'Condo - Common Wall', 'bedrooms': 1, 'bathrooms': 1.0, 'square_feet': 735, 'built': 1983, 'lot_size': nan, 'neighborhood': 'MCCormick Pier Building A', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97209', 'high_school': 'Lincoln', 'middle_school': 'Other', 'elementary_school': 'Chapman'}
{'address': '1902 NW 143RD AVE 5, Portland OR 97229', 'price': 209900, 'home_type': 'Condo - 1 Story', 'bedrooms': 2, 'bathrooms': 1.0, 'square_feet': 814, 'built': 1992, 'lot_size': nan, 'neighborhood': 'unknown', 'county': 'Washington', 'city': 'Portland', 'zipcode': '97229', 'high_school': 'Sunset', 'middle_school': 'Cedar Park', 'elementary_school': 'Terra Linda'}
{'address': '618 S NEVADA ST D, Portland OR 97219', 'price': 209900, 'home_type': 'Condo - Contemporary', 'bedrooms': 1, 'bathrooms': 1.0, 'square_feet': 664, 'built': 1997, 'lot_size': nan, 'neighborhood': 'CORBETT', 'county': 'Multnomah', 'c

{'address': '17803 SE CARUTHERS ST, Portland OR 97233', 'price': 229800, 'home_type': 'Single Family - Ranch', 'bedrooms': 3, 'bathrooms': 1.0, 'square_feet': 1032, 'built': 1959, 'lot_size': 0.16, 'neighborhood': 'unknown', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97233', 'high_school': 'Centennial', 'middle_school': 'Centennial', 'elementary_school': 'Happy Valley'}
{'address': '2829 SE BELMONT ST 104, Portland OR 97214', 'price': 229900, 'home_type': 'Condo - Mediterranean/Missio', 'bedrooms': 1, 'bathrooms': 1.0, 'square_feet': 698, 'built': 1927, 'lot_size': nan, 'neighborhood': 'BELMONT', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97214', 'high_school': 'Franklin', 'middle_school': 'Sunnyside Env', 'elementary_school': 'Sunnyside Env'}
{'address': '15041 NE SISKIYOU CT, Portland OR 97230', 'price': 229900, 'home_type': 'Condo - Detached Condo', 'bedrooms': 2, 'bathrooms': 2.0, 'square_feet': 1638, 'built': 1973, 'lot_size': nan, 'neighborhood': 'unknown'

{'address': '1640 SW MONTGOMERY ST D, Portland OR 97201', 'price': 244500, 'home_type': 'Condo - Craftsman', 'bedrooms': 1, 'bathrooms': 1.0, 'square_feet': 688, 'built': 2001, 'lot_size': nan, 'neighborhood': 'Portland State University', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97201', 'high_school': 'Lincoln', 'middle_school': 'Other', 'elementary_school': 'Chapman'}
{'address': '141 N HAYDEN BAY DR, Portland OR 97217', 'price': 244777, 'home_type': 'Condo - Contemporary', 'bedrooms': 1, 'bathrooms': 1.0, 'square_feet': 905, 'built': 1991, 'lot_size': nan, 'neighborhood': 'HAYDEN ISLAND / JANTZEN B', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97217', 'high_school': 'Jefferson', 'middle_school': 'Other', 'elementary_school': 'Faubion'}
{'address': '7922 SE 76TH PL, Portland OR 97206', 'price': 244900, 'home_type': 'Single Family - Traditional', 'bedrooms': 3, 'bathrooms': 2.0, 'square_feet': 1456, 'built': 1981, 'lot_size': 0.17, 'neighborhood': 'unknown', 'c

...

-------------
53 invalid literal for int() with base 10: '1300000 - 1400000'
{'address': '3023 SW SCHOLLS FERRY RD, Portland OR 97221', 'price': 1400000, 'home_type': 'Single Family - Traditional', 'bedrooms': 3, 'bathrooms': 6.0, 'square_feet': 4045, 'built': 1978, 'lot_size': 0.71, 'neighborhood': 'unknown', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97221', 'high_school': 'Lincoln', 'middle_school': 'West Sylvan', 'elementary_school': 'Bridlemile'}
{'address': '7440 SE 34TH AVE, Portland OR 97202', 'price': 1400000, 'home_type': 'Single Family - Craftsman', 'bedrooms': 5, 'bathrooms': 3.0, 'square_feet': 4645, 'built': 1924, 'lot_size': 0.17, 'neighborhood': 'Eastmoreland', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97202', 'high_school': 'Cleveland', 'middle_school': 'Sellwood', 'elementary_school': 'Duniway'}
{'address': '5000 NW 140TH AVE, Portland OR 97229', 'price': 1400000, 'home_type': 'Single Family - Contemporary', 'bedrooms': 5, 'bathrooms': 4.0, 's

{'address': '3410 NE BEAKEY ST, Portland OR 97212', 'price': 1495000, 'home_type': 'Single Family - English', 'bedrooms': 5, 'bathrooms': 4.0, 'square_feet': 3621, 'built': 1936, 'lot_size': 0.17, 'neighborhood': 'ALAMEDA / BEAUMONT', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97212', 'high_school': 'Grant', 'middle_school': 'Beaumont', 'elementary_school': 'Alameda'}
{'address': '12150 NW MAPLE HILL LN, Portland OR 97229', 'price': 1495000, 'home_type': 'Single Family - Mid-Century Modern', 'bedrooms': 4, 'bathrooms': 5.0, 'square_feet': 4703, 'built': 1966, 'lot_size': 1.06, 'neighborhood': 'unknown', 'county': 'Washington', 'city': 'Portland', 'zipcode': '97229', 'high_school': 'Sunset', 'middle_school': 'Cedar Park', 'elementary_school': 'Bonny Slope'}
{'address': '2895 SW West Point AVE, Portland OR 97225', 'price': 1499900, 'home_type': 'Single Family - Farmhouse', 'bedrooms': 4, 'bathrooms': 4.0, 'square_feet': 3784, 'built': 2020, 'lot_size': 0.33, 'neighborhood': '

{'address': '1306 SW MILITARY RD, Portland OR 97219', 'price': 1695000, 'home_type': 'Single Family - Traditional', 'bedrooms': 4, 'bathrooms': 4.0, 'square_feet': 4924, 'built': 2001, 'lot_size': 1.22, 'neighborhood': 'unknown', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97219', 'high_school': 'Riverdale', 'middle_school': 'Riverdale', 'elementary_school': 'Riverdale'}
{'address': '3911 NW LEWIS LN, Portland OR 97229', 'price': 1695000, 'home_type': 'Single Family - Custom Style', 'bedrooms': 4, 'bathrooms': 6.0, 'square_feet': 6454, 'built': 2006, 'lot_size': 0.26, 'neighborhood': 'FOREST HEIGHTS', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97229', 'high_school': 'Lincoln', 'middle_school': 'West Sylvan', 'elementary_school': 'Forest Park'}
{'address': '7566 SW CEDARCREST ST, Portland OR 97225', 'price': 1699000, 'home_type': 'Single Family - Traditional', 'bedrooms': 6, 'bathrooms': 6.0, 'square_feet': 8094, 'built': 2010, 'lot_size': 0.9, 'neighborhood': 'un

{'address': '2448 SW COMMONWEALTH AVE, Portland OR 97201', 'price': 1950000, 'home_type': 'Single Family - Traditional', 'bedrooms': 4, 'bathrooms': 6.0, 'square_feet': 5884, 'built': 1989, 'lot_size': 0.23, 'neighborhood': 'unknown', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97201', 'high_school': 'Lincoln', 'middle_school': 'West Sylvan', 'elementary_school': 'Ainsworth'}
{'address': '1075 NW NORTHRUP ST 2115, Portland OR 97209', 'price': 1995000, 'home_type': 'Condo - Contemporary', 'bedrooms': 2, 'bathrooms': 3.0, 'square_feet': 1777, 'built': 2016, 'lot_size': nan, 'neighborhood': 'Pearl District / Cosmopol', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97209', 'high_school': 'Lincoln', 'middle_school': 'West Sylvan', 'elementary_school': 'Chapman'}
{'address': '1225 S MARY FAILING DR, Portland OR 97219', 'price': 1995000, 'home_type': 'Single Family - Traditional', 'bedrooms': 6, 'bathrooms': 6.0, 'square_feet': 5792, 'built': 1959, 'lot_size': 1.03, 'neigh

{'address': '1150 NW Quimby ST 2108, Portland OR 97209', 'price': 3016000, 'home_type': 'Condo - Contemporary', 'bedrooms': 2, 'bathrooms': 3.0, 'square_feet': 2701, 'built': 2018, 'lot_size': nan, 'neighborhood': 'PEARL DISTRICT / VISTA', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97209', 'high_school': 'Lincoln', 'middle_school': 'West Sylvan', 'elementary_school': 'Chapman'}
{'address': '1150 NW Quimby ST 2112, Portland OR 97209', 'price': 3185000, 'home_type': 'Condo - Contemporary', 'bedrooms': 2, 'bathrooms': 3.0, 'square_feet': 2715, 'built': 2018, 'lot_size': nan, 'neighborhood': 'PEARL DISTRICT / VISTA', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97209', 'high_school': 'Lincoln', 'middle_school': 'West Sylvan', 'elementary_school': 'Chapman'}
{'address': '1500 SW 11TH AVE 2701, Portland OR 97201', 'price': 3200000, 'home_type': 'Condo - Contemporary', 'bedrooms': 3, 'bathrooms': 3.0, 'square_feet': 2920, 'built': 2007, 'lot_size': nan, 'neighborhood': '

In [9]:
# Create a dataframe
housing_data_df = pd.DataFrame(list_home_dict)
housing_data_df.drop_duplicates(inplace=True)
housing_data_df.head()

,address,price,home_type,bedrooms,bathrooms,square_feet,built,lot_size,neighborhood,county,city,zipcode,high_school,middle_school,elementary_school
0,"3157 NE MARINE DR, Portland OR 97035",65000,Floating Home - 1 Story,1,1.0,800,1964,NaN,unknown,Multnomah,Portland,97035,Current Price:,Jefferson,Faubion
1,"17452 NE GLISAN ST #7, Portland OR 97230",72000,Manufactured - Double Wide Manufact,2,2.0,1152,1988,NaN,unknown,Multnomah,Portland,97230,Reynolds,Reynolds,Hartley
2,"9034 SE 78TH PL, Portland OR 97206",79950,Manufactured - Double Wide Manufact,3,2.0,1344,1997,NaN,unknown,Clackamas,Portland,97206,Current Price:,Milwaukie,Whitman
3,"16000 SE POWELL BLVD 75, Portland OR 97236",79950,Manufactured - Double Wide Manufact,3,2.0,1404,1990,NaN,unknown,Multnomah,Portland,97236,Centennial,Centennial,Powell Butte
4,"12846 SE RAMONA ST 6, Portland OR 97236",93900,Manufactured - Double Wide Manufact,3,2.0,1297,1997,NaN,unknown,Multnomah,Portland,97236,David Douglas,Alice Ott,Gilbert Hts


In [11]:
# Check data collected verse data missed
print(f'Listings: {len(housing_data_df)}')
print(f'Missed Listings: {len(loading_error_links)}')

Listings: 1781
Missed Listings: 55


# Save Data

In [13]:
# Saving housing data
housing_data_df.to_csv("../Resources/housingData.csv", index = False, header = True)

# Saving list of housing links
with open("../Resources/housing_links.txt", "wb") as fp:   #Pickling
    pickle.dump(link_list, fp)